In [1]:
import os
os.chdir('/home/evgenii/work/itmo/DroneRescue')
import numpy as np
from agent.llm_agent import LLMAgent
from environment.environment import Environment
from environment.constants import OBSTACLE_MAP, OBJECT_MAP, COLOR_MAP

In [2]:
AREA_SIZE = (100, 100)
NUM_TARGETS = 3
NUM_OBSTACLES = 50
OBSERVATION_AREA = (9,9)

import os
api_key = os.getenv("OPENAI_API_KEY")
agent = LLMAgent(area_size=AREA_SIZE, object_map=OBJECT_MAP, observation_area=OBSERVATION_AREA, api_key=api_key, chat_gpt_model="gpt-4o-2024-08-06")

In [3]:
agent_position = (0,0)
env = Environment(obstacles_map=OBSTACLE_MAP,
                color_map=COLOR_MAP,
                object_map=OBJECT_MAP,
                area_size=AREA_SIZE,
                num_targets=NUM_TARGETS,
                num_obstacles=NUM_OBSTACLES
                )
observation_area_of_agent = agent.observation_area
new_position, done, observation, obs_binary_mask, metadata = env.step(4, agent_position, observation_area_of_agent)
agent_position = new_position
env.render_env('./exp_1/environment.jpg')

# Update visited map
agent.update_visited_map(observation, obs_binary_mask)

# Add current agent position to the visited map
visited_map_wiht_cut_agent_position = agent.visited_map
visited_map_wiht_cut_agent_position[agent_position] = 4 

# Add current agent position to the local observation
observation_with_cur_agent_position = observation.copy()
observation_with_cur_agent_position[metadata["agent_position_in_local_observation"]] = 4 


for step in range(10):
    # print(visited_map_wiht_cut_agent_position)
    response = agent.generate_action_by_model(visited_map_wiht_cut_agent_position, observation_with_cur_agent_position)
    action = response["action"]
    explaination = response["explaination"]
    print(f"Step {step}: {action}. Explainataion: {explaination}")

    new_position, done, observation, obs_binary_mask, metadata = env.step(action, agent_position, observation_area_of_agent)
    agent_position = new_position

    # Update visited map
    agent.update_visited_map(observation, obs_binary_mask)

    # Add current agent position to the visited map
    visited_map_wiht_cut_agent_position = agent.visited_map
    visited_map_wiht_cut_agent_position[agent_position] = 4 

    # Add current agent position to the local observation
    observation_with_cur_agent_position = observation.copy()
    observation_with_cur_agent_position[metadata["agent_position_in_local_observation"]] = 4 

    env.render_visited_map(visited_map_wiht_cut_agent_position, f'./exp_1/visited_map_{step}.jpg')
    env.render_observation(observation_with_cur_agent_position, f'./exp_1/observation_{step}.jpg')

Step 0: 3. Explainataion: The drone is currently at the top-left corner of the observation area. No obstacles or target points are visible within the current observation area. The best course of action is to move right to explore further and potentially find the target or more unvisited cells.
Step 1: 1. Explainataion: The drone's current position is at the top row, and moving down will allow the drone to explore unvisited areas below while avoiding obstacles.
Step 2: 3. Explainataion: The current position of the drone is marked by '4' in the observation grid. The immediate surroundings in the observation grid are filled with '0', indicating background. This means there are no obstacles around the current position. The drone should move right or down to explore new areas, but let's choose to move right first to maximize the exploration in that direction.
Step 3: 3. Explainataion: The drone is currently positioned at the center of the observation grid. Since there are no obstacles or ta

In [19]:
response["explanaition"]

'The current observation shows that the entire top half of the observation area contains obstacles (1s). The bottom half of the observation area is clear (0s), indicating a possible path. Moving down will allow the drone to explore this unvisited area.'

In [ ]:
"""'The current observation shows that the entire
 top half of the observation area 
 contains obstacles (1s). The bottom
   half of the observation area is 
   clear (0s), indicating a possible 
   path. Moving down will allow the drone to
     explore this unvisited area.'
     """

SyntaxError: incomplete input (3866867020.py, line 1)